In [ ]:
# !sudo apt-get -qq -y update
# !sudo apt-get -qq -y install python3-pip
# !sudo apt-get -qq -y install python-is-python3
# !wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz \
#   && tar -zxvf flatc.tar.gz \
#   && sudo chmod +x flatc \
#   && sudo mv flatc /usr/bin/
# !pip install -qq -U pip \
#   && pip install -qq tensorflow \
#   && pip install -qq -U onnx==1.14.1 \
#   && python -m pip install -qq onnx_graphsurgeon \
#         --index-url https://pypi.ngc.nvidia.com \
#   && pip install -qq -U onnxruntime==1.16.0 \
#   && pip install -qq-U onnxsim==0.4.33 \
#   && pip install -qq -U  simple_onnx_processing_tools \
#   && pip install -qq -U onnx2tf \
#   && pip install -qq -U protobuf==3.20.3 \
#   && pip install -qq -U h5py==3.7.0 \
#   && pip install --qq U psutil==5.9.5 \
#   && pip install -qq -U ml_dtypes==0.2.0 \
#   && pip install -qq --user --upgrade openvino2tensorflow

In [63]:
# !wget https://github.com/PINTO0309/onnx2tf/releases/download/0.0.2/resnet18-v1-7.onnx

# # INT8 Quantization (per-channel)
# $ onnx2tf -i emotion-ferplus-8.onnx -oiqt
# # INT8 Quantization (per-tensor)
# $ onnx2tf -i emotion-ferplus-8.onnx -oiqt -qt per-tensor\

!rm -rf ../models/saved_model/


!sor4onnx -if ../models/best.onnx -of ../models/best_cleaned.onnx --old_new "onnx::" ""  --mode full --search_mode prefix_match


# import onnx
# import re

def alphanumeric_name(name):
    """Convert a string into its alphanumeric version."""
    return ''.join(e for e in name if e.isalnum())
import onnx
from onnx import shape_inference

def alphanumeric_name(name):
    """Convert a string into its alphanumeric version."""
    return ''.join(e for e in name if e.isalnum())

def clean_onnx_model(model_path, output_path):
    # Load the model
    model = onnx.load(model_path)

    # Rename nodes
    for node in model.graph.node:
        if not node.name.isalnum():
            node.name = alphanumeric_name(node.name)

    # Rename value_info (intermediate tensor names)
    for value_info in model.graph.value_info:
        if not value_info.name.isalnum():
            value_info.name = alphanumeric_name(value_info.name)

    # Rename inputs and outputs if needed
    for input in model.graph.input:
        if not input.name.isalnum():
            input.name = alphanumeric_name(input.name)
    for output in model.graph.output:
        if not output.name.isalnum():
            output.name = alphanumeric_name(output.name)

    # Infer shapes and other properties after modifying the graph
    inferred_model = shape_inference.infer_shapes(model)

    # Save the cleaned model
    onnx.save(inferred_model, output_path)

clean_onnx_model("../models/best_cleaned.onnx", "../models/best_cleaned_1.onnx")



# # Simple node deletion tool for onnx.
# from soa4onnx import outputs_add

# onnx_graph = outputs_add(
#     input_onnx_file_path="../models/best_cleaned_1.onnx",
#     output_op_names=["/model.24/Concat_output_0", "/model.24/Concat_1_output_0","/model.24/Concat_2_output_0"],
#     output_onnx_file_path="../models/best_cleaned_2.onnx",
# )

# !snd4onnx --remove_node_names output0 --input_onnx_file_path ../models/best_cleaned_2.onnx  --output_onnx_file_path ../models/best_pre_nms.onnx


INFO: Finish!


In [58]:
# Very simple NCHW and NHWC conversion tool for ONNX. Change to the specified input order for each and every input OP.
#  Also, change the channel order of RGB and BGR. Simple Channel Converter for ONNX. Simple Channel Conversion for ONNX.

!scc4onnx --input_onnx_file_path ../models/best_cleaned_pre_nms.onnx --output_onnx_file_path ../models/best_cleaned_pre_nms.onnx   \
--input_op_names_and_order_dims images [0,2,3,1] --channel_change_inputs images 1


INFO: Finish!


In [60]:
!onnx2tf -i ../models/best_cleaned_pre_nms.onnx -ois images:1,3,640,640  #oiqt -qt per-tensor


Model optimizing started ============================================================
Traceback (most recent call last):
  File "/home/gokkulnath/Mosquito_detect/.env/lib/python3.10/site-packages/onnx2tf/onnx2tf.py", line 606, in convert
    result = subprocess.check_output(
  File "/usr/lib/python3.10/subprocess.py", line 421, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
  File "/usr/lib/python3.10/subprocess.py", line 526, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['onnxsim', '../models/best_cleaned_pre_nms.onnx', '../models/best_cleaned_pre_nms.onnx', '--overwrite-input-shape', 'images:1,3,640,640']' returned non-zero exit status 1.


Automatic generation of each OP name started ========================================
Automatic generation of each OP name complete!

Model loaded ========================================================================

Model conversion started ==========